In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cardiovascular_Disease"
cohort = "GSE235307"

# Input paths
in_trait_dir = "../../input/GEO/Cardiovascular_Disease"
in_cohort_dir = "../../input/GEO/Cardiovascular_Disease/GSE235307"

# Output paths
out_data_file = "../../output/preprocess/Cardiovascular_Disease/GSE235307.csv"
out_gene_data_file = "../../output/preprocess/Cardiovascular_Disease/gene_data/GSE235307.csv"
out_clinical_data_file = "../../output/preprocess/Cardiovascular_Disease/clinical_data/GSE235307.csv"
json_path = "../../output/preprocess/Cardiovascular_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression and atrial fibrillation prediction"
!Series_summary	"The aim of this study was to identify a blood gene expression profile that predicts atrial fibrillation in heart failure patients"
!Series_overall_design	"Cardiac blood samples were obtained from the coronary sinus during CRT-D (Cardiac Resynchronization Therapy - Defibrillator) placement in heart failure patients. Patients were followed during 1 year."
Sample Characteristics Dictionary:
{0: ['tissue: Whole blood'], 1: ['gender: Male', 'gender: Female'], 2: ['age: 63', 'age: 60', 'age: 72', 'age: 66', 'age: 70', 'age: 64', 'age: 61', 'age: 44', 'age: 54', 'age: 50', 'age: 79', 'age: 51', 'age: 55', 'age: 67', 'age: 52', 'age: 73', 'age: 76', 'age: 43', 'age: 68', 'age: 78', 'age: 69', 'age: 57', 'age: 59', 'age: 53', 'age: 65', 'age: 56', 'age: 74', 'age: 38', 'age: 71', 'age: 37'], 3: ['cardiopathy: ischemic', 'cardiopathy: non ischemic', 'cardiopathy: mixed'], 4: ['cardiac rhyt

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the dataset description, this appears to be gene expression data from blood samples
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: The trait appears to be "Atrial fibrillation" which can be inferred from row 5
# "cardiac rhythm after 1 year follow-up: Sinus rhythm" or "cardiac rhythm after 1 year follow-up: Atrial fibrillation"
trait_row = 5

# For age: Age data is available in row 2
age_row = 2

# For gender: Gender data is available in row 1
gender_row = 1

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert atrial fibrillation status to binary (0: No AF, 1: AF)."""
    if pd.isna(value) or value is None:
        return None
    value = value.strip() if isinstance(value, str) else value
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "Atrial fibrillation" in value:
        return 1  # Atrial fibrillation is present (positive case)
    elif "Sinus rhythm" in value:
        return 0  # Normal sinus rhythm (negative case)
    return None

def convert_age(value):
    """Convert age to continuous numeric value."""
    if pd.isna(value) or value is None:
        return None
    value = value.strip() if isinstance(value, str) else value
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0: Female, 1: Male)."""
    if pd.isna(value) or value is None:
        return None
    value = value.strip() if isinstance(value, str) else value
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "Male" in value or value.lower() == "male":
        return 1
    elif "Female" in value or value.lower() == "female":
        return 0
    return None

# 3. Save Metadata - Initial Filtering
# Trait data availability is determined by whether trait_row is None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load clinical data
    clinical_df_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_df_path):
        clinical_data = pd.read_csv(clinical_df_path)
        
        # Extract clinical features
        clinical_features_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted features
        preview = preview_df(clinical_features_df)
        print("Preview of clinical features:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save to CSV
        clinical_features_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical features saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Cardiovascular_Disease/GSE235307/GSE235307_series_matrix.txt.gz


Gene data shape: (58717, 119)
First 20 gene/probe identifiers:
Index(['4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '20', '21', '22', '23'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examine the gene identifiers
# These appear to be numeric identifiers (4, 5, 6, etc.) which are not standard human gene symbols
# Standard human gene symbols would be like BRCA1, TP53, etc.
# Therefore, these identifiers need to be mapped to proper gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'COL', 'ROW', 'NAME', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'LOCUSLINK_ID', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE']
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'GB_ACC': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'LOCUSLINK_ID': [nan, nan, nan, 50865.0, 23704.0], 'GENE_SYMBOL': [nan, nan, nan, 'HEBP1', 'KCNE4'], 'GENE_NAME': [nan, nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4'], 'UNIGENE_ID': [nan, nan, nan, 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe the gene identifier and gene symbol columns in the gene annotation
# From the preview, we can see:
# - The gene identifiers in gene_data are numeric IDs (like '4', '5', '6')
# - In gene_annotation, the 'ID' column contains similar numeric identifiers
# - The 'GENE_SYMBOL' column contains the human gene symbols like 'HEBP1', 'KCNE4'

# 2. Extract the gene identifier and gene symbol columns
gene_mapping_df = get_gene_mapping(
    annotation=gene_annotation,
    prob_col='ID',  # The column with probe IDs matching gene_data index
    gene_col='GENE_SYMBOL'  # The column with human gene symbols
)

print("Gene mapping preview:")
print(preview_df(gene_mapping_df, n=5))
print(f"Shape of gene mapping dataframe: {gene_mapping_df.shape}")

# 3. Apply gene mapping to convert from probe-level to gene-level expression
gene_data = apply_gene_mapping(
    expression_df=gene_data,
    mapping_df=gene_mapping_df
)

print("\nGene expression data after mapping:")
print(f"Shape of gene expression data: {gene_data.shape}")
print("First 10 gene symbols:")
print(list(gene_data.index[:10]))

# Apply gene symbol normalization to standardize symbols and aggregate duplicates
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nGene expression data after normalization:")
print(f"Shape of gene expression data after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(list(gene_data.index[:10]))

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene expression data to the specified output file
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Gene mapping preview:
{'ID': ['4', '5', '6', '7', '8'], 'Gene': ['HEBP1', 'KCNE4', 'BPIFA3', 'LOC100129869', 'IRG1']}
Shape of gene mapping dataframe: (54295, 2)



Gene expression data after mapping:
Shape of gene expression data: (20353, 119)
First 10 gene symbols:
['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']

Gene expression data after normalization:
Shape of gene expression data after normalization: (19847, 119)
First 10 normalized gene symbols:
['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06', 'AAA1']



Gene expression data saved to ../../output/preprocess/Cardiovascular_Disease/gene_data/GSE235307.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols  
# (Note: We already normalized in step 6, but let's explicitly ensure it's done properly)

# 2. Load the clinical data and extract features using the correct trait_row and conversion functions from Step 2
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Define correct conversion functions matching the ones in Step 2
def convert_trait(value):
    """Convert atrial fibrillation status to binary (0: No AF, 1: AF)."""
    if pd.isna(value) or value is None:
        return None
    value = value.strip() if isinstance(value, str) else value
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "Atrial fibrillation" in value:
        return 1  # Atrial fibrillation is present (positive case)
    elif "Sinus rhythm" in value:
        return 0  # Normal sinus rhythm (negative case)
    return None

def convert_age(value):
    """Convert age to continuous numeric value."""
    if pd.isna(value) or value is None:
        return None
    value = value.strip() if isinstance(value, str) else value
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0: Female, 1: Male)."""
    if pd.isna(value) or value is None:
        return None
    value = value.strip() if isinstance(value, str) else value
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "Male" in value or value.lower() == "male":
        return 1
    elif "Female" in value or value.lower() == "female":
        return 0
    return None

# Extract clinical features using the correct row indices from Step 2
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data, 
    trait=trait, 
    trait_row=5,  # Correctly using cardiac rhythm row as identified in Step 2
    convert_trait=convert_trait,
    age_row=2,  # Age information from row 2
    convert_age=convert_age,
    gender_row=1,  # Gender information from row 1
    convert_gender=convert_gender
)

print(f"Clinical features shape: {clinical_features.shape}")
print("Clinical features preview:")
print(preview_df(clinical_features))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# 4. Handle missing values
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "Dataset contains gene expression data from cardiac blood samples of heart failure patients, with atrial fibrillation status tracked over 1 year."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Clinical features shape: (3, 119)
Clinical features preview:
{'GSM7498589': [0.0, 63.0, 1.0], 'GSM7498590': [0.0, 60.0, 1.0], 'GSM7498591': [0.0, 60.0, 1.0], 'GSM7498592': [0.0, 72.0, 1.0], 'GSM7498593': [0.0, 63.0, 1.0], 'GSM7498594': [0.0, 66.0, 0.0], 'GSM7498595': [0.0, 70.0, 1.0], 'GSM7498596': [0.0, 64.0, 1.0], 'GSM7498597': [0.0, 63.0, 1.0], 'GSM7498598': [0.0, 61.0, 1.0], 'GSM7498599': [0.0, 70.0, 0.0], 'GSM7498600': [0.0, 64.0, 1.0], 'GSM7498601': [0.0, 63.0, 1.0], 'GSM7498602': [0.0, 44.0, 1.0], 'GSM7498603': [0.0, 54.0, 1.0], 'GSM7498604': [0.0, 44.0, 1.0], 'GSM7498605': [0.0, 50.0, 1.0], 'GSM7498606': [1.0, 79.0, 1.0], 'GSM7498607': [0.0, 63.0, 1.0], 'GSM7498608': [0.0, 63.0, 0.0], 'GSM7498609': [1.0, 64.0, 1.0], 'GSM7498610': [0.0, 60.0, 1.0], 'GSM7498611': [0.0, 51.0, 1.0], 'GSM7498612': [0.0, 55.0, 1.0], 'GSM7498613': [0.0, 55.0, 1.0], 'GSM7498614': [1.0, 67.0, 1.0], 'GSM7498615': [0.0, 52.0, 1.0], 'GSM7498616': [0.0, 70.0, 0.0], 'GSM7498617': [0.0, 54.0, 1.0], 'GSM749861

Linked data shape after handling missing values: (119, 19850)
For the feature 'Cardiovascular_Disease', the least common label is '1.0' with 13 occurrences. This represents 10.92% of the dataset.
The distribution of the feature 'Cardiovascular_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 55.0
  50% (Median): 63.0
  75%: 68.0
Min: 37.0
Max: 79.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 32 occurrences. This represents 26.89% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Cardiovascular_Disease/GSE235307.csv
